<a href="https://colab.research.google.com/github/mwithgod3952/Conquering-Kaggle-s-challenge/blob/main/Marketing_Analytics_Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size = 4>Data preparation</font>

In [1]:
# Basic environment
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    ITALIC = '\x1B'

# drive
from google.colab import drive; drive.mount('/content/drive', force_remount=True)

# path
import os

org_pth = '/content/drive/MyDrive'
_new_pth = [x for x in os.listdir(org_pth) if x.startswith('Cohort')][0]
new_pth = f'{org_pth}/{_new_pth}' 

if new_pth.startswith('/content') and new_pth.endswith('practice'):
    os.chdir(new_pth)
    wp = os.getcwd()
    print(f'\t\n {new_pth}')

Mounted at /content/drive
	
 /content/drive/MyDrive/Cohort_Analysis_practice


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files; files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jhleekr","key":"740259ff05a1aa87d388c6ecd1153f10"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d carrie1/ecommerce-data
!kaggle datasets download -d mustafacicek/online-retail-final

print('\n')

!unzip ecommerce-data
!unzip online-retail-final

  0% 0.00/7.20M [00:00<?, ?B/s]
100% 7.20M/7.20M [00:00<00:00, 121MB/s]
  0% 0.00/6.30M [00:00<?, ?B/s]
100% 6.30M/6.30M [00:00<00:00, 154MB/s]


Archive:  ecommerce-data.zip
  inflating: data.csv                
Archive:  online-retail-final.zip
  inflating: online_retail_final.csv  


In [2]:
dataset = [x for x in os.listdir(wp) if x.endswith('.csv')]
print(dataset)

['data.csv', 'online_retail_final.csv']


<font size = 4>Library</font>

In [102]:
import re
import ast
import random
import numpy as np
import pandas as pd

from google.colab import data_table

In [4]:
df1 = pd.read_csv(f'{wp}/{dataset[0]}', encoding= 'unicode_escape')
df1.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
print(f'\t\n {color.BOLD}Shape : {df1.shape}{color.END} \n')

print(f'{color.BOLD}- NULL{color.END}')
print(df1.isnull().sum())
print('\n')

	
 Shape : (541909, 8) 

- NULL
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64




In [6]:
df1[df1.Description.isnull()].iloc[:2, :]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom


In [7]:
des_nullDf = df1[df1.Description.isnull()].reset_index( drop=True )

In [8]:
if len(des_nullDf) == des_nullDf.CustomerID.isnull().sum():
    print('T')
else:
    print('F')

T


In [9]:
des_nullDf.UnitPrice.value_counts()

0.0    1454
Name: UnitPrice, dtype: int64

<font size = 2>상위의 과정을 통해 "Description" 열에 포함된 내용이 없을 때 "CustomerID" 정보 역시 누락되어 있음을 확인하였습니다, 더불어, "UnitPrice"는 0임을 확하였습니다.</font>

In [10]:
des_notnullDf = df1[df1.Description.notnull()].reset_index( drop=True )
ID_NULL_cnt = des_notnullDf.CustomerID.isnull().sum()
print(f'\n\t {color.BOLD}- "Description"정보가 확인되나, ID 정보가 누락된 경우의 수 : {ID_NULL_cnt}{color.END}')


	 - "Description"정보가 확인되나, ID 정보가 누락된 경우의 수 : 133626


In [11]:
print(f'\n\t - {color.BOLD}"StockCode"정보는 정수로만 이루어져 있거나 정수와 함께 문자열이 결합되어 기록된 것이 일반적이고, 이 외 주식코드 검색을 목적으로 문자열로만 이루어진 것들이 아래와 같이 존재합니다.{color.END}\n')
for i in df1[df1['StockCode'].apply(lambda x: str('Y') if re.search('[0-9]+', x) else str('N')) == 'N'].StockCode.unique():
    print(f'\t\t {i}')


	 - "StockCode"정보는 정수로만 이루어져 있거나 정수와 함께 문자열이 결합되어 기록된 것이 일반적이고, 이 외 주식코드 검색을 목적으로 문자열로만 이루어진 것들이 아래와 같이 존재합니다.

		 POST
		 D
		 DOT
		 M
		 BANK CHARGES
		 S
		 AMAZONFEE
		 m
		 DCGSSBOY
		 DCGSSGIRL
		 PADS
		 B
		 CRUK


In [12]:
df1 = df1[df1.Description.notnull() & df1.CustomerID.notnull()]
df1.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [13]:
df1.loc[df1['InvoiceNo'].apply(lambda x: str('Y') if re.search('[a-zA-Z]+', x) else str('N')) == 'Y', 'InvoiceNo'].apply(lambda x: str(x)[0]).unique()

array(['C'], dtype=object)

In [14]:
wC  = df1[df1['InvoiceNo'].apply(lambda x: x.startswith('C'))]
nwC = df1[df1['InvoiceNo'].apply(lambda x: True if len(set('AC') & set(x)) == 0 else False)]
if (len(wC) == sum(wC.Quantity < 0)) and (len(nwC) == sum(nwC.Quantity >= 0)):
    print('T')
else:
    print('N')    

T


<font size = 2>'InvoiceNO'정보 값 중 특정 값에 알파벳 값이 결합된 경우를 발견하여 확인해본 결과, 일반 정수값 번호 외에 'C'와 'A'가 결합된 경우를 확인하였습니다.</font>

<font size = 2>그 중, 'C'로 구분한 Quantity값이 모두 음수 인 것을 확인하였고 더불어, Quantity값이 음수이나 'InvoiceNO'값이 잘못 구분된 케이스 역시 없음으로, 취소된 주문건을 구분하기 위한 구분자임을 확신할 수 있습니다.</font>


<font size = 2>우선 'Cancelled'라는 열을 만들어 주문취소가 포함된 행을 1로 나머지 행을 0으로 구분합니다.</font>


In [15]:
df1.loc[wC.index, 'Cancelled'] = int(1); df1.loc[df1.Cancelled.isnull(), 'Cancelled'] = int(0)

<font size = 2>아래부터는 주문건에 대한 정보만을을 취합하여 분석을 진행하겠습니다.</font>

In [16]:
_df1 = df1[df1.Cancelled == 0]
_df1.Cancelled.value_counts()

0.0    397924
Name: Cancelled, dtype: int64

In [135]:
col_relevance = []
contain_alpha = _df1[_df1['StockCode'].str.contains("^[a-zA-Z]")]
stc_str = contain_alpha.iloc[:, 1].value_counts().index.tolist()
for i in range(len(stc_str)):
    y = set(list(map(lambda x: str(x), _df1[_df1['StockCode'] == stc_str[i]].loc[:, ['StockCode', 'Description']].values.tolist())))
    col_relevance.append(ast.literal_eval(list(y)[0]))

cr_df = pd.DataFrame( columns=list(_df1.columns)[1:] )   
for i in range(2):
    cr_df.iloc[:, i] = list(zip(*col_relevance))[i]
for j in range(len(stc_str)):
    cr_df.iloc[j, 2:] = random.sample(_df1[_df1.StockCode == stc_str[j]].iloc[:, 3:].values.tolist(), 1)[0] 

print(f'{color.BOLD}- number of values ​​containing the alphabet{color.END}')
print(contain_alpha.StockCode.value_counts())
print(f'{color.BLUE}-{color.END}'*3)
print(f'{color.BOLD}- comparison table comparing the values ​​of the Stockcode and other columns{color.END}')
cr_df    

- number of values ​​containing the alphabet
POST            1099
M                290
C2               133
DOT               16
BANK CHARGES      12
PADS               4
Name: StockCode, dtype: int64
---
- comparison table comparing the values ​​of the Stockcode and other columns


,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
0,POST,POSTAGE,1,12/2/2011 10:24,18.0,12569.0,Germany,0.0
1,M,Manual,2,12/14/2010 10:38,0.21,17894.0,United Kingdom,0.0
2,C2,CARRIAGE,1,2/18/2011 11:36,50.0,14911.0,EIRE,0.0
3,DOT,DOTCOM POSTAGE,1,10/24/2011 17:07,638.3,14096.0,United Kingdom,0.0
4,BANK CHARGES,Bank Charges,1,12/2/2010 15:08,15.0,15823.0,United Kingdom,0.0
5,PADS,PADS TO MATCH ALL CUSHIONS,1,7/26/2011 10:13,0.001,15618.0,United Kingdom,0.0


<font size = 2></font>

In [146]:
for i in range(6):
    print(_df1[_df1.StockCode == stc_str[i]])
    print()

       InvoiceNo StockCode Description  Quantity      InvoiceDate  UnitPrice  \
45        536370      POST     POSTAGE         3   12/1/2010 8:45       18.0   
386       536403      POST     POSTAGE         1  12/1/2010 11:27       15.0   
1123      536527      POST     POSTAGE         1  12/1/2010 13:04       18.0   
5073      536840      POST     POSTAGE         1  12/2/2010 18:27       18.0   
5258      536852      POST     POSTAGE         1   12/3/2010 9:51       18.0   
...          ...       ...         ...       ...              ...        ...   
541198    581493      POST     POSTAGE         1  12/9/2011 10:10       15.0   
541216    581494      POST     POSTAGE         2  12/9/2011 10:13       18.0   
541730    581570      POST     POSTAGE         1  12/9/2011 11:59       18.0   
541767    581574      POST     POSTAGE         2  12/9/2011 12:09       18.0   
541768    581578      POST     POSTAGE         3  12/9/2011 12:16       18.0   

        CustomerID      Country  Cancel